# Facial Recognition Software

In [ ]:
#1. Generate dataset 
#2. Train the classifier and save it 
#3. Detect the face and name it if its already stored in our dataset

# Generate dataset

In [ ]:
import cv2
import os
import glob
import numpy as np
from PIL import Image

In [ ]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y + h, x:x + w]
        return cropped_face

    cap = cv2.VideoCapture(0)
    directory = "data"
    if not os.path.exists(directory):  # Check if directory exists
        os.makedirs(directory)  # Create directory if it doesn't exist

    # Find the highest user ID in the filenames of the existing images
    files = glob.glob(f"{directory}/user_*.jpg")
    if files:
        user_id = max(int(file.split('_')[1].split('.')[0]) for file in files)
    else:
        user_id = 0

    user_id += 1  # Increment the user ID for the new images

    images_generated = 0  # New variable to count the number of images generated

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            images_generated += 1  # Increment the count each time an image is generated
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = os.path.join(directory, f"user_{user_id}_{images_generated}.jpg")
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(images_generated), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow("Cropped face", face)
            if cv2.waitKey(1) == 13 or images_generated == 50:  # Check if 50 images have been generated
                break
    cap.release()
    cv2.destroyAllWindows()
    print(f"Collecting samples for user {user_id} is completed..............")

generate_dataset()

# Train the classifier and save it

In [ ]:
import os 
import numpy as np
from PIL import Image

In [ ]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')  # Convert image to grayscale
        imageNp = np.array(img, 'uint8')        
        
        # Split filename and handle errors
        id = os.path.split(image)[1].split('.')[1]  # Extract ID from filename

        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)
    
    #Tain the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()